## Accessing the project directory on my Google Drive

In [1]:
import os
from google.colab import drive

drive.mount('/drive', force_remount=False)

Mounted at /drive


In [2]:
project_dir = "/drive/My Drive/RNN_seq2seq"

# change working directory to project_dir
os.chdir(project_dir)

## Dependencies

In [3]:
import random
import pandas as pd 
from string import ascii_lowercase
from IPython.display import clear_output

from os import makedirs
from os.path import join

import torch.nn as nn
import torch.optim as optim

from scripts.utils import read_datasets, save_dict_as_json
from scripts.dataloader import get_text_encoder_decoder, customize_dataloader_func
from scripts.visualization import plot_training_log, plot_performances_per_seq_len
from scripts.pytorch_utils import get_model, count_parameters, train_and_evaluate, evaluate, get_results

## Log time and random seeds

In [4]:
from datetime import datetime
import random 
import numpy as np
import torch

now = datetime.now()
print("Time stamp:", now.strftime("%Y-%m-%d %H:%M:%S"))

seed = 3266473
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

Time stamp: 2023-06-03 05:00:53


## Data loading and processing

In [5]:
task = "identity"
train, dev, test, gen = read_datasets(f"data/{task}")

In [6]:
len(train), len(dev), len(test), len(gen)

(10000, 10000, 50000, 100000)

In [7]:
in_vocab = ascii_lowercase
out_vocab = ascii_lowercase
in_seq_encoder, in_seq_decoder = get_text_encoder_decoder(in_vocab)
out_seq_encoder, out_seq_decoder = get_text_encoder_decoder(out_vocab)

In [8]:
dataloader_func = customize_dataloader_func(in_seq_encoder, 
                                            out_seq_encoder, 
                                            padding_idx=1, 
                                            batch_size=1000, 
                                            shuffle=False)

train_dl = dataloader_func(train)
dev_dl = dataloader_func(dev)
test_dl = dataloader_func(test, batch_size=5000)
gen_dl = dataloader_func(gen, batch_size=5000)

## Experiments

In [9]:
task_name = f"{task}_follow_up"

metadata = []
main_results = []
task_folder = join("experiments", task_name)
meta_col = ["RNN", "Attention", "lr", "Hidden size", "Embedding size", "Param #", "Max Epoch #"]
main_res_col = ["Run #", "RNN", "Attention", "Dataset", "Loss", 
                "Full Sequence Accuracy", "First N-symbol Accuracy", "Overlap Rate"]


for rnn_type in ["SRNN", "GRU", "LSTM"]:
    for hs in [64, 32, 16]:
        for run_num in range(1, 4):
            
            print(f"RNN type: {rnn_type}; Hidden size: {hs}; run num: {run_num}\n")
            
            use_attn = True
            embd_dim = 128
            hidden_size = hs
            learning_rate = 5e-4
            weight_decay = 1e-5
            max_epoch_num = 500
            
            ModelConfig = {"rnn_type": rnn_type,
                           "embd_dim": embd_dim,
                           "hidden_size": hidden_size,
                           "device": "cuda",
                           "num_layers": 1,
                           "dropout_rate": 0.0,
                           "use_attention": use_attn,
                           "bidirectional": False,
                           "in_vocab_size": len(in_vocab) + 2, # for the two boundary symbols 
                           "out_vocab_size": len(out_vocab) + 2, # for the two boundary symbols 
                           "reduction_method": "sum"}
            
            model = get_model(ModelConfig)
            
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

            folder = join(task_folder, rnn_type, f"HiddenSize-{hs}", str(run_num))
            makedirs(folder, exist_ok=True)
            
            saved_model_fp = join(folder, "model.pt")
            log = train_and_evaluate(model, train_dl, dev_dl, 
                                     criterion, optimizer, 
                                     saved_model_fp=saved_model_fp, 
                                     acc_threshold=-0.1, 
                                     print_eval_freq=10, 
                                     max_epoch_num=max_epoch_num, 
                                     train_exit_acc=0.9999, 
                                     eval_exit_acc=0.995, 
                                     teacher_forcing_ratio=1.0)
            
            save_dict_as_json(log, join(folder, "training_log.json"))
            save_dict_as_json(ModelConfig, join(folder, "ModelConfig.json"))
            plot_training_log(log, show_plot=False, saved_plot_fp=join(folder, "training_plot.png"))
            
            train_res, dev_res = get_results(log, train_log=True)
            
            _, perfs_train = evaluate(model, train_dl, criterion, 
                                      per_seq_len_performance=True)
            _, perfs_dev = evaluate(model, dev_dl, criterion, 
                                      per_seq_len_performance=True)
            
            test_aggr, perfs_test = evaluate(model, test_dl, criterion, 
                                             per_seq_len_performance=True)
            
            gen_aggr, perfs_gen = evaluate(model, gen_dl, criterion, 
                                             per_seq_len_performance=True)
            
            test_res = get_results(test_aggr, train_log=False)
            gen_res = get_results(gen_aggr, train_log=False)
            
            for res, ds in zip([train_res, dev_res, test_res, gen_res], ["Train", "Dev", "Test", "Gen"]):
                main_results.append([run_num, rnn_type, use_attn, ds] + res)
                       
            
            save_dict_as_json(perfs_train, join(folder, "perfs_train.json"))
            save_dict_as_json(perfs_dev, join(folder, "perfs_dev.json"))
            save_dict_as_json(perfs_test, join(folder, "perfs_test.json"))
            save_dict_as_json(perfs_gen, join(folder, "perfs_gen.json"))
            plot_performances_per_seq_len(perfs_train, False, join(folder, "perfs_train.png"))
            plot_performances_per_seq_len(perfs_dev, False, join(folder, "perfs_dev.png"))
            plot_performances_per_seq_len(perfs_test, False, join(folder, "perfs_test.png"))
            plot_performances_per_seq_len(perfs_gen, False, join(folder, "perfs_gen.png"))
            
            clear_output(wait=True)
        
        param_num = count_parameters(model)
        metadata.append([rnn_type, use_attn, learning_rate, hidden_size, embd_dim, param_num, max_epoch_num])

pd.DataFrame(metadata, columns=meta_col).to_csv(join(task_folder, "metadata.csv"), index=False)
pd.DataFrame(main_results, columns=main_res_col).to_csv(join(task_folder, "main_results.csv"), index=False)

RNN type: LSTM; Hidden size: 16; run num: 3

The model has 27,900 trainable parameters
Current epoch: 10, 
training performance: {'loss': 3.23150053024292, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.0038751089188589194, 'overlap rate': 0.11534588106338106}
evaluation performance: {'loss': 3.232506251335144, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.00421767662892663, 'overlap rate': 0.11673631479631479}

Current epoch: 20, 
training performance: {'loss': 3.1210489749908445, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.0068102263014763005, 'overlap rate': 0.14047803571428572}
evaluation performance: {'loss': 3.122585129737854, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.006407082084582084, 'overlap rate': 0.1391677493339993}

Current epoch: 30, 
training performance: {'loss': 3.062968635559082, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.007565902430902433, 'overlap rate': 0.14972244852369854}
evaluation p

## Automatically disconnect and delete the runtime 

In [ ]:
from google.colab import runtime
runtime.unassign()